In [1]:
%matplotlib widget
%matplotlib inline
import sys
from sys import platform
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.ticker import ScalarFormatter
import numpy as np
import pickle
import seaborn as sns
import scipy
import emcee
import corner
from IPython.display import display, Math
from tqdm import tqdm
from multiprocessing import Pool

from astropy.io import fits, ascii
from astropy.table import Table
from astropy.modeling import functional_models, fitting
from astropy.timeseries import LombScargle

import stingray.events as ev
import stingray.lightcurve as lc
from stingray import io
import stingray.powerspectrum as powspec 
import stingray.crossspectrum as crossspec
from hendrics.efsearch import dyn_folding_search, z_n_search, folding_search
import stingray.gti as sting_gti
import stingray.pulse.pulsar as plsr
from stingray import stats


sns.set_context('talk')
# sns.set_style("whitegrid")
sns.set_palette("colorblind")

sys.path.insert(1, '/Users/sean/scripts/helpers')

from stingray_plus import *

def minimize_remainder(arr, min_div, max_div):
    divisors = np.linspace(min_div, max_div, num=100)
    remainders = []
    for div in divisors:
        remainders.append(np.sum(np.mod(arr, div)))
        
    return divisors[np.argmin(remainders)]

def power_law(f, B, gamma):
    return B*np.power(f,gamma)

def Lorentzian(f, peakf, Q, A):
    # gamma = HWHM
    # peakf = centroid frequency
    gamma = peakf/(2.0 * Q)
    return (A * np.square(gamma)/(np.pi*gamma*(np.square(f-peakf) + np.square(gamma))))

def zero_center_Lorentzian(f, gamma, A):
    # gamma = HWHM
    # peakf = centroid frequency
    return (A * np.square(gamma)/(np.pi*gamma*(np.square(f) + np.square(gamma))))

def Lorentzian_C(f, peakf, Q, A, C):
    return Lorentzian(f, peakf, Q, A) + C

def Lorentzian_power(f, peakf, Q, A, B, gamma):
    return Lorentzian(f, peakf, Q, A) + power_law(f, B, gamma)

def N_Lorentzian(f, *args):
    N = int(len(args)/3)
    peak_nu = args[:N]
    Qs = args[N:N+N]
    As = args[N+N:N+N+N]
    model = np.zeros(np.shape(f))
    for i in range(N):
        model = model + Lorentzian(f, peak_nu[i], Qs[i], As[i])
        
    return model

def N_Lorentzian_C(f, *args):
    N = int((len(args)-1)/3)
    peak_nu = args[:N]
    Qs = args[N:N+N]
    As = args[N+N:N+N+N]
    C = args[-1]
    model = C * np.ones(np.shape(f))
    for i in range(N):
        model = model + Lorentzian(f, peak_nu[i], Qs[i], As[i])
        
    return model

def N_Lorentzian_power(f, *args):
    N = int((len(args)-2)/3)
    peak_nu = args[:N]
    Qs = args[N:N+N]
    As = args[N+N:N+N+N]
    B, alpha = args[-2:]
    model = power_law(f, B, alpha)
    for i in range(N):
        gamma = peak_nu[i]/(2.0 * Qs[i])
        model = model + (As[i] * np.square(gamma)/(np.pi*gamma*(np.square(f-peak_nu[i]) + np.square(gamma))))
        
    return model

def QPO_scan(cross_spec, f_min=1e-4, f_max=2000., f_bin=1000, n_lorentz = 1):
    f_mask = (cross_spec.freq > f_min) * (cross_spec.freq < f_max)
    freq_steps = np.logspace(np.log10(cross_spec.freq[f_mask][0]), np.log10(cross_spec.freq[f_mask][-1]), f_bin + 2)
    xdata = cross_spec.freq[f_mask]
    ydata = cross_spec.power[f_mask]
    sigma = cross_spec.power_err[f_mask]
    
    pl_popt, pl_pcov = scipy.optimize.curve_fit(power_law, xdata, ydata, sigma = sigma, p0 = [10., -1.0], \
                                                bounds=np.array([(0.0, np.inf), (-np.inf, 0.0)]).T)
    print(pl_popt)
    chisq0 = np.sum(((ydata - power_law(xdata, *pl_popt)) / sigma) ** 2)
    chisq = []
    for i in tqdm(range(len(freq_steps[1:-1]))):
        f = freq_steps[i+1]
        popt, pcov = scipy.optimize.curve_fit(Lorentzian_power, xdata, ydata, sigma = sigma, p0 = [f, 2.0, 0.1, pl_popt[0], pl_popt[1]], \
                                              bounds=np.array([(f - (f-freq_steps[i])/2., f + (freq_steps[i+2] - f)/2.0), (1.0,np.inf), (0.0,np.inf), (0.0, np.inf), (-np.inf, 0.0)]).T)
        chisq.append(np.sum(((ydata - Lorentzian_power(xdata, *popt)) / sigma) ** 2))
    dof = len(xdata)-len(popt)
    return freq_steps[1:-1], chisq0, np.array(chisq), dof



def fit_peaks(xdata, ydata, sigma, nu_peak):
    
    popt_arr = []
    pcov_arr = []

    for i, p in enumerate(nu_peak):
        f_bound = None
        if len(nu_peak)==1:
            f_bound = (np.min(xdata), np.max(xdata))
        else:
            if i == 0:
                f_bound = (np.min(xdata), p + (nu_peak[i+1] - p)/2)
            elif i==len(nu_peak)-1:
                f_bound = (p + (nu_peak[i-1] - p)/2, np.max(xdata))
            else:
                f_bound = (p + (nu_peak[i-1] - p)/2, p + (nu_peak[i+1] - p)/2)

        par_bounds = np.array([f_bound, (1.0,np.inf), (0, np.inf), (0, np.inf), (-np.inf, 0.0)]).T
        p0 = [p, 5.0, 0.1, 0.02, -0.5]
        popt, pcov = scipy.optimize.curve_fit(Lorentzian_power, xdata, ydata, sigma = sigma, p0 = p0, bounds = par_bounds)
        popt_arr.append(popt)
        pcov_arr.append(pcov)

    popt_arr = np.array(popt_arr)
    pcov_arr = np.array(pcov_arr)

    return popt_arr, pcov_arr

def get_rms(events, centroids, radius, PI_min=35, PI_max=1210, nu_min=1e-4, nu_max=100., split_time=512, bin_time = 1/1024, plot = False):
    
    curve_A = events[0].to_lc(dt = bin_time, pi_low=PI_min, pi_high=PI_max, centroid = centroids[0], radius = radius)
    curve_B = events[1].to_lc(dt = bin_time, pi_low=PI_min, pi_high=PI_max, centroid = centroids[1], radius = radius)
    
    average_CPDS = crossspec.AveragedCrossspectrum(curve_A, curve_B, segment_size=split_time, norm = 'frac')
    
    if plot:
        f_res = 0.1
        plt.figure(figsize=(9,6))
        averaged_cross_log = average_CPDS.rebin_log(f=f_res)
        averaged_cross_log_err = average_CPDS.df*np.power(1.+f_res, range(len(averaged_cross_log.freq)))/2.
        plt.errorbar(averaged_cross_log.freq, averaged_cross_log.power*averaged_cross_log.freq, xerr=averaged_cross_log_err, yerr=averaged_cross_log.power_err*averaged_cross_log.freq, fmt='none', lw=0.5)
        plt.xscale('log')
        plt.ylim((1e-4,5*np.max(averaged_cross_log.power.real)))
        plt.yscale('log')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Leahy Power')
        plt.ylabel(r'$\mathrm{\nu P_{\nu}\ (rms/mean)^{2}}$')
        plt.tight_layout()
        plt.show()
#         plt.savefig(plot_dir + 'averaged_cross_spectrum_' + str(int(split_time)) + 's.pdf')
        plt.close()
    
    rms_square = np.sum(average_CPDS.power[(average_CPDS.freq > nu_min) * (average_CPDS.freq < nu_max)])*average_CPDS.df
    rms_square_err = np.sqrt(np.sum(np.square(average_CPDS.power_err[(average_CPDS.freq > nu_min) * (average_CPDS.freq < nu_max)])))*average_CPDS.df
    
    if rms_square < 0.0:
        uplim = True
        rms = np.sqrt(rms_square + 2.6*rms_square_err)
    else:
        uplim = False
        rms = np.sqrt(rms_square)
        
    rms_err = np.sqrt(np.sum(np.square(average_CPDS.power_err[(average_CPDS.freq > nu_min) * (average_CPDS.freq < nu_max)])))*average_CPDS.df/(2*rms)

    return rms, rms_err, uplim

def model_continuum_noise_zero_center(cpds, plot=True, plot_dir='/Users/sean/Desktop/', f_res = 0.5):

    chisq0 = np.sum(((cpds.power-np.mean(cpds.power))/ cpds.power_err) ** 2)

    popt, pcov = scipy.optimize.curve_fit(zero_center_Lorentzian, cpds.freq, cpds.power, sigma = cpds.power_err, \
                                          p0 = [0.1, 10.], bounds= [[0.0,0.0], [np.inf, np.inf]])
    chisq = np.sum(((cpds.power - zero_center_Lorentzian(cpds.freq, *popt)) / cpds.power_err) ** 2)

    if plot:
        cpds_log = cpds.rebin_log(f=f_res)
        temp_err = cpds.df*np.power(1.+f_res, range(len(cpds_log.freq)))/2.

        plt.figure(figsize=(9,6))
        plt.errorbar(cpds_log.freq, cpds_log.power*cpds_log.freq, xerr=temp_err, \
                     yerr=cpds_log.power_err*cpds_log.freq, fmt='none', lw=0.5, color='black')
        plt.step(np.concatenate([cpds_log.freq-temp_err, [cpds_log.freq[-1]+temp_err[-1]]]), \
                 np.concatenate([cpds_log.power*cpds_log.freq, [(cpds_log.power*cpds_log.freq)[-1]]]), where='post', color='black', lw=0.5)
        
        plt.plot(cpds.freq,zero_center_Lorentzian(cpds.freq, *popt)*cpds.freq, color='red', lw=1.0)
        plt.xscale('log')
        plt.ylim((1e-6,3*np.max(cpds_log.power.real*cpds_log.freq)))
        plt.yscale('log')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel(r'$\mathrm{\nu P_{\nu}\ (rms/mean)^{2}}$')
        plt.tight_layout()
        plt.savefig(plot_dir + 'CPDS_nuPnu_continuum_zero_center.pdf')
        plt.close()

        plt.figure(figsize=(9,6))
        plt.errorbar(cpds_log.freq, cpds_log.power, xerr=temp_err, \
                     yerr=cpds_log.power_err, fmt='none', lw=0.5, color='black')
        plt.step(np.concatenate([cpds_log.freq-temp_err, [cpds_log.freq[-1]+temp_err[-1]]]), \
                 np.concatenate([cpds_log.power, [cpds_log.power[-1]]]), where='post', color='black', lw=0.5)
        
        plt.plot(cpds.freq,zero_center_Lorentzian(cpds.freq, *popt), color='red', lw=1.0)
        plt.xscale('log')
        plt.ylim((1e-6,3*np.max(cpds_log.power.real)))
        plt.yscale('log')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel(r'$\mathrm{Power\ (rms/mean)^{2}/Hz}$')
        plt.tight_layout()
        plt.savefig(plot_dir + 'CPDS_Pnu_continuum_zero_center.pdf')
        plt.close()
    
    return popt, pcov, chisq0, chisq



def model_continuum_noise(cpds, plot=True, plot_dir='/Users/sean/Desktop/', f_res = 0.5):
    n_L = 0
    L_args = []
    L_bounds = []
    chisq0 = np.sum(((cpds.power-np.mean(cpds.power))/ cpds.power_err) ** 2)
    chisq = []
    popt_list = []
    pcov_list = []
    del_chisq = -100000
    while del_chisq < 0.0:
        n_L += 1
        if n_L ==1:
            L_args = np.array([1.0, 0.1, 10.])
            L_bounds = np.array([[[0.0, np.max(cpds.freq)], [0.0, 2.0], [0,np.inf]]])
        else:
            L_args = np.vstack((L_args, [1.0, 0.1, 10.]))
            L_bounds = np.concatenate((L_bounds, [[(0.0, np.max(cpds.freq)), (0.0, 10.0), (0,np.inf)]]))
                
        temp_p0 = L_args.T.flatten()
        temp_bounds = [L_bounds.T.flatten()[:3*n_L], L_bounds.T.flatten()[3*n_L:]]
        
        popt, pcov = scipy.optimize.curve_fit(N_Lorentzian, cpds.freq, cpds.power, sigma = cpds.power_err, \
                                              p0 = temp_p0, bounds= temp_bounds)
        temp_chisq = np.sum(((cpds.power - N_Lorentzian(cpds.freq, *popt)) / cpds.power_err) ** 2)
        
        if n_L==1:
            del_chisq = temp_chisq-chisq0
        else:
            del_chisq = temp_chisq-chisq[-1]
        chisq.append(temp_chisq)
        popt_list.append(popt)
        pcov_list.append(pcov)
        
        L_args = np.array([popt[:n_L], popt[n_L:n_L + n_L], popt[n_L+n_L:n_L+n_L+n_L]]).T

        if plot:
            cpds_log = cpds.rebin_log(f=f_res)
            temp_err = cpds.df*np.power(1.+f_res, range(len(cpds_log.freq)))/2.

            plt.figure(figsize=(9,6))
            plt.errorbar(cpds_log.freq, cpds_log.power*cpds_log.freq, xerr=temp_err, \
                         yerr=cpds_log.power_err*cpds_log.freq, fmt='none', lw=0.5, color='black')
            plt.step(np.concatenate([cpds_log.freq-temp_err, [cpds_log.freq[-1]+temp_err[-1]]]), \
                     np.concatenate([cpds_log.power*cpds_log.freq, [(cpds_log.power*cpds_log.freq)[-1]]]), where='post', color='black', lw=0.5)
            
            for i in range(n_L):
                plt.plot(cpds.freq, Lorentzian(cpds.freq, *((L_args[i])))*cpds.freq, color='red', ls='dotted', lw=1.0)
            plt.plot(cpds.freq,N_Lorentzian(cpds.freq, *popt)*cpds.freq, color='red', lw=1.0)
            plt.xscale('log')
            plt.ylim((1e-6,3*np.max(cpds_log.power.real*cpds_log.freq)))
            plt.yscale('log')
            plt.xlabel('Frequency (Hz)')
            plt.ylabel(r'$\mathrm{\nu P_{\nu}\ (rms/mean)^{2}}$')
            plt.tight_layout()
            plt.savefig(plot_dir + 'CPDS_nuPnu_continuum_' + str(int(n_L)) + '_comps.pdf')
            plt.close()
            
            plt.figure(figsize=(9,6))
            plt.errorbar(cpds_log.freq, cpds_log.power, xerr=temp_err, \
                         yerr=cpds_log.power_err, fmt='none', lw=0.5, color='black')
            plt.step(np.concatenate([cpds_log.freq-temp_err, [cpds_log.freq[-1]+temp_err[-1]]]), \
                     np.concatenate([cpds_log.power, [cpds_log.power[-1]]]), where='post', color='black', lw=0.5)

            for i in range(n_L):
                plt.plot(cpds.freq, Lorentzian(cpds.freq, *((L_args[i]))), color='red', ls='dotted', lw=1.0)
            plt.plot(cpds.freq,N_Lorentzian(cpds.freq, *popt), color='red', lw=1.0)
            plt.xscale('log')
            plt.ylim((1e-6,3*np.max(cpds_log.power.real)))
            plt.yscale('log')
            plt.xlabel('Frequency (Hz)')
            plt.ylabel(r'$\mathrm{Power\ (rms/mean)^{2}/Hz}$')
            plt.tight_layout()
            plt.savefig(plot_dir + 'CPDS_Pnu_continuum_' + str(int(n_L)) + '_comps.pdf')
            plt.close()
    
    return n_L, popt_list, pcov_list, chisq0, chisq





/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/crossspectrum.py:21: UserWarning: Using standard scipy fft
  warnings.warn("Using standard scipy fft")
/Users/sean/opt/anaconda3/lib/python3.8/site-packages/hendrics/io.py:24: UserWarning: Warning! NetCDF is not available. Using pickle format.
  warnings.warn(msg)


## Setup

In [2]:
# OBSID_list = [str(80502324000 + int(2*(i+1))) for i in range(8)]
# print(OBSID_list)

OBSID_list = ['80502324008', '80502324010', '80502324012', '80502324014']

if platform=='linux' or platform=='linux2':
    print('Working on SRL server')
    root_dir = '/disk/lif2/spike/MAXI_J0637-430/'
elif platform=='darwin':
    print('Working on Macbook')
    root_dir = '/Volumes/Samsung_1TB/AstroData/MAXI_J0637-430/'


Working on Macbook


In [3]:
cpds_list = []
plt.ion()
bin_time = 1/512
split_time = 256
buff = False
lc_buff_size = 60
bkg_bin = 32
noise_params = []
noise_error = []
# cnt_rate = []
# cnt_rate_err = []
# softness_ratio = []
# softness_ratio_err = []
for OBSID in OBSID_list:
    print(OBSID)
    
    ### Setup
    products_dir = root_dir + OBSID + '_products/'
    plot_dir = root_dir + 'figures/' + OBSID + '/'
    plt.ion()
    PI_min = 35     # 3.0 keV
    # PI_min = 260     # 12.0 keV
    # PI_max = 960   # 40.0 keV
    PI_max = 1909   # 78.0 keV
    
    events = extract_events(products_dir + 'nu' + OBSID + 'A01_cl_barycorr.evt', \
                products_dir + 'nu' + OBSID + 'B01_cl_barycorr.evt')
    
    bkg_curve_A = nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'A01_bk.lc', buff = buff, buffersize = lc_buff_size)
    bkg_curve_B = nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'B01_bk.lc', buff = buff, buffersize = lc_buff_size)
#     
#     bkg_curve_total = sum_lc(bkg_curve_A, bkg_curve_B)
    
    curve_A = bkg_subtract(nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'A01_sr.lc', buff = buff, buffersize = lc_buff_size), \
                           bkg_curve_A, bkg_bin=bkg_bin)
    curve_B = bkg_subtract(nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'B01_sr.lc', buff = buff, buffersize = lc_buff_size),\
                           bkg_curve_B, bkg_bin=bkg_bin)
    curve_total = sum_lc(curve_A, curve_B)
#     curve_10s = curve_total.rebin(dt_new=10)

    centroid_A = curve_A.centroid
    centroid_B = curve_B.centroid
    extraction_radius = curve_A.radius
    area_ratio = np.square(curve_A.radius/bkg_curve_A.radius)
    
    bkg_rate = bkg_curve_A.meanrate
    src_rate = curve_A.meanrate


#     print(centroid_A)
#     print(centroid_B)
#     print(extraction_radius)
#     cnt_rate.append(np.mean(curve_total.countrate))
#     cnt_rate_err.append(np.std(curve_total.countrate))
    

    t_start = np.min(curve_total.time)

    ### Plot the lightcurve
#     plt.figure(figsize = (9,6))
#     plt.errorbar(curve_10s.time-t_start, curve_10s.countrate, xerr=curve_10s.dt/2., yerr=curve_10s.countrate_err, fmt='none', lw = 0.5)
#     plt.xlabel('Time (s)')
#     plt.ylabel('NuSTAR count rate')
#     plt.tight_layout()
#     plt.savefig(plot_dir + 'bkg_sub_lightcurve.pdf')
#     plt.close()
    
    ### Create CPDS
    curve_A = events[0].to_lc(dt = bin_time, pi_low=PI_min, pi_high=PI_max, centroid = centroid_A, radius = extraction_radius)
    curve_B = events[1].to_lc(dt = bin_time, pi_low=PI_min, pi_high=PI_max, centroid = centroid_B, radius = extraction_radius)
    
# #     curve_A_soft = events[0].to_lc(dt = 1, pi_low=35, pi_high=110, centroid = centroid_A, radius = extraction_radius)
# #     curve_B_soft = events[1].to_lc(dt = 1, pi_low=35, pi_high=110, centroid = centroid_B, radius = extraction_radius)
# #     curve_A_hard = events[0].to_lc(dt = 1, pi_low=110, pi_high=260, centroid = centroid_A, radius = extraction_radius)
# #     curve_B_hard = events[1].to_lc(dt = 1, pi_low=110, pi_high=260, centroid = centroid_B, radius = extraction_radius)
    
#     soft_counts = np.sum((np.sqrt(np.square(events[0].x-centroid_A[0]) + np.square(events[0].y-centroid_A[1])) <= extraction_radius) * (events[0].pi >= 35) * (events[0].pi < 110)) + \
#                   np.sum((np.sqrt(np.square(events[1].x-centroid_B[0]) + np.square(events[1].y-centroid_B[1])) <= extraction_radius) * (events[1].pi >= 35) * (events[1].pi < 110))
#     hard_counts = np.sum((np.sqrt(np.square(events[0].x-centroid_A[0]) + np.square(events[0].y-centroid_A[1])) <= extraction_radius) * (events[0].pi >= 110) * (events[0].pi < 260)) + \
#                   np.sum((np.sqrt(np.square(events[1].x-centroid_B[0]) + np.square(events[1].y-centroid_B[1])) <= extraction_radius) * (events[1].pi >= 110) * (events[1].pi < 260))
    
#     softness_ratio.append(soft_counts/hard_counts)
#     softness_ratio_err.append(np.sqrt(soft_counts + (np.square(soft_counts)/hard_counts))/hard_counts)

    averaged_cross = crossspec.AveragedCrossspectrum(curve_A, curve_B, segment_size=split_time, norm = 'frac')
    averaged_cross.power = averaged_cross.power * np.square((1/(1-0.0025*(curve_A.meanrate))) * ((src_rate + (bkg_rate*area_ratio))/(src_rate)))
    averaged_cross.power_err = averaged_cross.power_err * np.square((1/(1-0.0025*(curve_A.meanrate))) * ((src_rate + (bkg_rate*area_ratio))/(src_rate)))

    
    cpds_list.append(averaged_cross)

    ### Plot CPDS
    f_res = 0.1
    averaged_cross_log = averaged_cross.rebin_log(f=f_res)
    averaged_cross_log_err = averaged_cross.df*np.power(1.+f_res, range(len(averaged_cross_log.freq)))/2.

    plt.figure(figsize=(9,6))
    plt.errorbar(averaged_cross_log.freq, averaged_cross_log.power*averaged_cross_log.freq, xerr=averaged_cross_log_err, yerr=averaged_cross_log.power_err*averaged_cross_log.freq, fmt='none', color='black', lw=0.5)
    plt.step(np.concatenate([averaged_cross_log.freq-averaged_cross_log_err, [averaged_cross_log.freq[-1]+averaged_cross_log_err[-1]]]), \
             np.concatenate([averaged_cross_log.power*averaged_cross_log.freq, [(averaged_cross_log.power*averaged_cross_log.freq)[-1]]]), where='post', color='black', lw=0.5)
    plt.xscale('log')
    plt.ylim((1e-4,10))
    plt.yscale('log')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel(r'$\mathrm{\nu P_{\nu}\ (rms/mean)^{2}}$')
    plt.tight_layout()
    # plt.show()
    plt.savefig(plot_dir + 'CPDS_nuPnu_' + str(int(split_time)) + 's.pdf')
    plt.close()

    plt.figure(figsize=(9,6))
    plt.errorbar(averaged_cross_log.freq, averaged_cross_log.power, xerr=averaged_cross_log_err, yerr=averaged_cross_log.power_err, fmt='none', color='black', lw=0.5)
    plt.step(np.concatenate([averaged_cross_log.freq-averaged_cross_log_err, [averaged_cross_log.freq[-1]+averaged_cross_log_err[-1]]]), \
             np.concatenate([averaged_cross_log.power, [averaged_cross_log.power[-1]]]), where='post', color='black', lw=0.5)
    plt.xscale('log')
    plt.ylim((1e-4,10))
    plt.yscale('log')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel(r'$\mathrm{Power\ (rms/mean)^{2}/Hz}$')
    plt.tight_layout()
    # plt.show()
    plt.savefig(plot_dir + 'CPDS_Pnu_' + str(int(split_time)) + 's.pdf')
    plt.close()
    
#     fig, (ax1, ax2) = plt.subplots(1, 2,  gridspec_kw = {'width_ratios':[1, 1]}, figsize=(20, 6))
    
#     ax1.errorbar(averaged_cross_log.freq, averaged_cross_log.power, xerr=averaged_cross_log_err, yerr=averaged_cross_log.power_err, fmt='none', color='black', lw=0.5)
#     ax1.step(np.concatenate([averaged_cross_log.freq-averaged_cross_log_err, [averaged_cross_log.freq[-1]+averaged_cross_log_err[-1]]]), \
#              np.concatenate([averaged_cross_log.power, [averaged_cross_log.power[-1]]]), where='post', color='black', lw=0.5)
#     ax1.set_xscale('log')
#     ax1.set_ylim((1e-4,10))
#     ax1.set_yscale('log')
#     ax1.set_xlabel('Frequency (Hz)')
#     ax1.set_ylabel(r'$\mathrm{Power\ (rms/mean)^{2}/Hz}$')

#     ax2.errorbar(curve_10s.time-t_start, curve_10s.countrate, xerr=curve_10s.dt/2., yerr=curve_10s.countrate_err, fmt='none', lw = 0.5)
#     ax2.set_xlabel('Time (s)')
#     ax2.set_ylabel('NuSTAR count rate')
#     ax2.set_ylim((0,60))
#     plt.tight_layout()
#     # plt.show()
#     plt.savefig(plot_dir + 'CPDS_Pnu_' + str(int(split_time)) + 's_4gif.pdf')
#     plt.close()


    
#     ## Calculate Lomb-Scargle
#     frequency, power = LombScargle(curve_total.time-t_start, curve_total.countrate, dy=curve_total.countrate_err).autopower()
#     plt.figure(figsize=(9,6))
#     plt.plot(frequency, power, rasterized=True)
#     plt.xscale('log')
#     plt.xlabel('Frequency (Hz)')
#     plt.ylabel('Lomb-Scargle Power')
#     plt.tight_layout()
#     plt.savefig(plot_dir + 'lomb_scargle.pdf')

#     ## Calculate Lomb-Scargle
#     frequency, power = LombScargle(curve_total.time-t_start, np.ones(np.shape(curve_total.countrate)), fit_mean=False, center_data=False).autopower()
#     plt.figure(figsize=(9,6))
#     plt.plot(frequency, power, rasterized=True)
#     plt.xscale('log')
#     plt.xlabel('Frequency (Hz)')
#     plt.ylabel('Lomb-Scargle Power')
#     plt.tight_layout()
#     plt.savefig(plot_dir + 'lomb_scargle_window.pdf')

#     ## Calculate bkg Lomb-Scargle
#     frequency, power = LombScargle(bkg_curve_total.time-t_start, bkg_curve_total.countrate, dy=bkg_curve_total.countrate_err).autopower()
#     plt.figure(figsize=(9,6))
#     plt.plot(frequency, power, rasterized=True)
#     plt.xscale('log')
#     plt.xlabel('Frequency (Hz)')
#     plt.ylabel('Lomb-Scargle Power')
#     plt.tight_layout()
#     plt.savefig(plot_dir + 'lomb_scargle_bkg.pdf')

    
#     ## Fit several Lorentzians
#     n_L, popt_list, pcov_list, chisq0, chisq = model_continuum_noise(averaged_cross, plot=True, plot_dir=plot_dir, f_res=f_res)

#     del_chisq = np.array(chisq) - chisq0

#     plt.figure(figsize=(9,6))
#     plt.plot(np.array(range(n_L)) + 1, -del_chisq)
#     plt.xlabel('N_Lorentzians')
#     plt.ylabel(r'$\Delta\chi^{2}$')
#     # plt.yscale('log')
#     plt.tight_layout()
#     plt.savefig(plot_dir + 'fit_chisq.pdf')
#     plt.close()

    
    ### Fit one zero-centered Lorentzian
    popt, pcov, chisq0, chisq = model_continuum_noise_zero_center(averaged_cross, plot=True, plot_dir=plot_dir, f_res=f_res)

#     del_chisq = chisq - chisq0
#     print(del_chisq/chisq0)
    
    noise_params.append(popt)
    noise_error.append(pcov)
    



80502324008


/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/utils.py:119: UserWarning: SIMON says: Errorbars on cross spectra are not thoroughly tested. Please report any inconsistencies.
  warnings.warn("SIMON says: {0}".format(message), **kwargs)
179it [00:09, 19.29it/s]
179it [00:09, 19.67it/s]
179it [00:09, 18.77it/s]


80502324010


/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/utils.py:119: UserWarning: SIMON says: Errorbars on cross spectra are not thoroughly tested. Please report any inconsistencies.
  warnings.warn("SIMON says: {0}".format(message), **kwargs)
438it [00:25, 17.00it/s]
438it [00:23, 18.28it/s]
438it [00:23, 18.62it/s]


80502324012


/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/utils.py:119: UserWarning: SIMON says: Errorbars on cross spectra are not thoroughly tested. Please report any inconsistencies.
  warnings.warn("SIMON says: {0}".format(message), **kwargs)
188it [00:11, 16.32it/s]
188it [00:09, 19.90it/s]
188it [00:11, 16.93it/s]


80502324014


/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/utils.py:119: UserWarning: SIMON says: Errorbars on cross spectra are not thoroughly tested. Please report any inconsistencies.
  warnings.warn("SIMON says: {0}".format(message), **kwargs)
246it [00:12, 19.97it/s]
246it [00:14, 16.88it/s]
246it [00:15, 15.81it/s]


In [5]:
plt.ion()
plot_dir = root_dir + 'figures/'
bin_time = 1/512
split_time = 256
buff = False
lc_buff_size = 60
bkg_bin = 32
fig, axes = plt.subplots(2, 2, gridspec_kw = {'height_ratios':[1,1], 'width_ratios': [1,1], 'hspace':0, 'wspace':0}, figsize=(12,9), sharex=True, sharey=True)
axes = axes.flatten()
# fig, axes = plt.subplots(len(OBSID_list), 1, gridspec_kw = {'height_ratios':[1 for i in range(len(OBSID_list))], 'hspace':0}, figsize=(9,3*len(OBSID_list)), sharex=True)
for i, OBSID in enumerate(OBSID_list):
    print(OBSID)
    
    ### Setup
    products_dir = root_dir + OBSID + '_products/'
    PI_min = 35     # 3.0 keV
    # PI_min = 260     # 12.0 keV
    # PI_max = 960   # 40.0 keV
    PI_max = 1909   # 78.0 keV
    
    events = extract_events(products_dir + 'nu' + OBSID + 'A01_cl_barycorr.evt', \
                products_dir + 'nu' + OBSID + 'B01_cl_barycorr.evt')
    
    bkg_curve_A = nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'A01_bk.lc', buff = buff, buffersize = lc_buff_size)
    bkg_curve_B = nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'B01_bk.lc', buff = buff, buffersize = lc_buff_size)
#     
#     bkg_curve_total = sum_lc(bkg_curve_A, bkg_curve_B)
    
    curve_A = bkg_subtract(nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'A01_sr.lc', buff = buff, buffersize = lc_buff_size), \
                           bkg_curve_A, bkg_bin=bkg_bin)
    curve_B = bkg_subtract(nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'B01_sr.lc', buff = buff, buffersize = lc_buff_size),\
                           bkg_curve_B, bkg_bin=bkg_bin)
    curve_total = sum_lc(curve_A, curve_B)
#     curve_10s = curve_total.rebin(dt_new=10)

    centroid_A = curve_A.centroid
    centroid_B = curve_B.centroid
    extraction_radius = curve_A.radius
    
    bkg_rate = bkg_curve_A.meanrate
    src_rate = curve_A.meanrate
    
    area_ratio = np.square(curve_A.radius/bkg_curve_A.radius)
#     print(centroid_A)
#     print(centroid_B)
#     print(extraction_radius)
#     cnt_rate.append(np.mean(curve_total.countrate))
#     cnt_rate_err.append(np.std(curve_total.countrate))
    

    t_start = np.min(curve_total.time)

    ### Create CPDS
    curve_A = events[0].to_lc(dt = bin_time, pi_low=PI_min, pi_high=PI_max, centroid = centroid_A, radius = extraction_radius)
    curve_B = events[1].to_lc(dt = bin_time, pi_low=PI_min, pi_high=PI_max, centroid = centroid_B, radius = extraction_radius)
    
    averaged_cross = crossspec.AveragedCrossspectrum(curve_A, curve_B, segment_size=split_time, norm = 'frac')
    averaged_cross.power = averaged_cross.power * np.square((1/(1-0.0025*(curve_A.meanrate))) * ((src_rate + (bkg_rate*area_ratio))/(src_rate)))
    averaged_cross.power_err = averaged_cross.power_err * np.square((1/(1-0.0025*(curve_A.meanrate))) * ((src_rate + (bkg_rate*area_ratio))/(src_rate)))
    
    ### Plot CPDS
    f_res = 0.1
    averaged_cross_log = averaged_cross.rebin_log(f=f_res)
    averaged_cross_log_err = averaged_cross.df*np.power(1.+f_res, range(len(averaged_cross_log.freq)))/2.
    ### Fit one zero-centered Lorentzian
    popt, pcov, chisq0, chisq = model_continuum_noise_zero_center(averaged_cross, plot=False)



#     axes[i].errorbar(averaged_cross_log.freq, averaged_cross_log.power*averaged_cross_log.freq, xerr=averaged_cross_log_err, yerr=averaged_cross_log.power_err*averaged_cross_log.freq, fmt='none', color='black', lw=0.5)
#     axes[i].step(np.concatenate([averaged_cross_log.freq-averaged_cross_log_err, [averaged_cross_log.freq[-1]+averaged_cross_log_err[-1]]]), \
#              np.concatenate([averaged_cross_log.power*averaged_cross_log.freq, [(averaged_cross_log.power*averaged_cross_log.freq)[-1]]]), where='post', color='black', lw=0.5)
#     axes[i].plot(averaged_cross_log.freq, zero_center_Lorentzian(averaged_cross_log.freq, *popt)*averaged_cross_log.freq, color='red', lw=1.0)
#     axes[i].set_xscale('log')
#     axes[i].set_ylim((1e-4,10))
#     axes[i].set_yscale('log')

    axes[i].errorbar(averaged_cross_log.freq, averaged_cross_log.power, xerr=averaged_cross_log_err, yerr=averaged_cross_log.power_err, fmt='none', color='black', lw=0.5)
    axes[i].step(np.concatenate([averaged_cross_log.freq-averaged_cross_log_err, [averaged_cross_log.freq[-1]+averaged_cross_log_err[-1]]]), \
             np.concatenate([averaged_cross_log.power, [averaged_cross_log.power[-1]]]), where='post', color='black', lw=0.5)
    axes[i].plot(averaged_cross_log.freq, zero_center_Lorentzian(averaged_cross_log.freq, *popt), color='red', lw=1.0)
    axes[i].set_xscale('log')
    axes[i].set_xlim(1.1e-3,10)
    axes[i].set_ylim((2e-4,10))
    axes[i].set_yscale('log')
    axes[i].tick_params(axis='both',which='both', direction='in', bottom=True, top=True, left=True, right=True)
    axes[i].text(2e-1,1, OBSID)

# axes[-1].set_xlabel('Frequency (Hz)')
# fig.text(0.02, 0.5, r'$\mathrm{\nu P_{\nu}\ (rms/mean)^{2}}$', ha='center', va='center', rotation='vertical')
fig.text(0.03, 0.5, r'$\mathrm{P_{\nu}\ (rms/mean)^{2}/Hz}$', ha='center', va='center', rotation='vertical')
fig.text(0.5, 0.03, 'Frequency (Hz)', ha='center', va='center', rotation='horizontal')
# plt.tight_layout()
plt.savefig(plot_dir + 'CPDS_Pnu_' + str(int(split_time)) + 's_2x2panel.pdf')
# plt.savefig(plot_dir + 'CPDS_Pnu_' + str(int(split_time)) + 's_4panel.pdf')
plt.close()



    



80502324008


/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/utils.py:119: UserWarning: SIMON says: Errorbars on cross spectra are not thoroughly tested. Please report any inconsistencies.
  warnings.warn("SIMON says: {0}".format(message), **kwargs)
179it [00:09, 19.30it/s]
179it [00:09, 18.93it/s]
179it [00:08, 20.08it/s]


80502324010


/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/utils.py:119: UserWarning: SIMON says: Errorbars on cross spectra are not thoroughly tested. Please report any inconsistencies.
  warnings.warn("SIMON says: {0}".format(message), **kwargs)
438it [00:20, 21.12it/s]
438it [00:20, 21.10it/s]
438it [00:20, 21.54it/s]


80502324012


/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/utils.py:119: UserWarning: SIMON says: Errorbars on cross spectra are not thoroughly tested. Please report any inconsistencies.
  warnings.warn("SIMON says: {0}".format(message), **kwargs)
188it [00:11, 16.58it/s]
188it [00:10, 18.64it/s]
188it [00:10, 18.46it/s]


80502324014


/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/utils.py:119: UserWarning: SIMON says: Errorbars on cross spectra are not thoroughly tested. Please report any inconsistencies.
  warnings.warn("SIMON says: {0}".format(message), **kwargs)
246it [00:12, 20.11it/s]
246it [00:12, 20.45it/s]
246it [00:12, 19.63it/s]


In [4]:
# for i, OBSID in enumerate(OBSID_list):
#     print(OBSID)
#     for j in range(len(noise_params[i])):
#         print(noise_params[i][j])
#         print(np.sqrt(np.diag(noise_error[i][j])))
plot_dir = root_dir + 'figures/'
# print(noise_params)
gamma_list = []
gamma_err_list = []
rms_list = []
rms_err_list =[]
for i, OBSID in enumerate(OBSID_list):
    print(OBSID)
#     print(noise_params[i])
#     print(np.sqrt(np.diag(noise_error[i])))

    gamma = noise_params[i][0]
    A = noise_params[i][1]

    gamma_err = np.sqrt(np.diag(noise_error[i]))[0]
    A_err = np.sqrt(np.diag(noise_error[i]))[1]

    rms = np.sqrt(A/2)
    rms_err = A_err/(4*np.sqrt(A/2))
    
    print(gamma)
    print(gamma_err)
    print(rms*100)
    print(rms_err*100)
    
    gamma_list.append(gamma)
    gamma_err_list.append(gamma_err)
    rms_list.append(rms)
    rms_err_list.append(rms_err)
    
# plt.figure(figsize = (9,6))
# plt.errorbar(cnt_rate[3:7], gamma_list[3:7], xerr=cnt_rate_err[3:7], yerr=gamma_err_list[3:7], fmt='none', lw = 0.5)
# plt.xlabel('NuSTAR count rate')
# plt.ylabel('HWHM')
# plt.tight_layout()
# plt.savefig(plot_dir + 'countrate_gamma.pdf')
# plt.close()

# plt.figure(figsize = (9,6))
# plt.errorbar(cnt_rate[3:7], rms_list[3:7], xerr=cnt_rate_err[3:7], yerr=rms_err_list[3:7], fmt='none', lw = 0.5)
# plt.xlabel('NuSTAR count rate')
# plt.ylabel('RMS (%)')
# plt.tight_layout()
# plt.savefig(plot_dir + 'countrate_rms.pdf')
# plt.close()

# plt.figure(figsize = (9,6))
# plt.errorbar(softness_ratio[3:7], gamma_list[3:7], xerr=softness_ratio_err[3:7], yerr=gamma_err_list[3:7], fmt='none', lw = 0.5)
# plt.xlabel('Softness Ratio')
# plt.ylabel('HWHM')
# plt.tight_layout()
# plt.savefig(plot_dir + 'ratio_gamma.pdf')
# plt.close()

80502324008
0.6423934514312533
0.07488879024408482
26.956151039975794
1.1088762204150802
80502324010
0.10319106386323645
0.007932000866518385
31.11380743291693
0.8350629598886821
80502324012
0.06386755826066375
0.011157848069751116
31.757466566978
1.9214105688574192
80502324014
0.02853591188465425
0.008188798923152975
33.36458146516527
3.219448716090985


# RMS-Energy relation

In [44]:
keV_list = np.array([[3.0, 5.0], [5.0, 8.0], [8.0, 15.0], [15.0, 50.0]])
PI_list = eV_to_PI(keV_list*1000)
rms_list = []
rms_err_list = []
uplim_list = []

for x in PI_list:
    print(x)
    rms, rms_err, uplim = get_rms(events, [centroid_A, centroid_B], extraction_radius, PI_min=x[0], PI_max=x[1], nu_min=1e-4, nu_max=1., split_time=split_time, bin_time = bin_time, plot = False)
    rms_list.append(rms*100)
    rms_err_list.append(rms_err*100)
    uplim_list.append(uplim)
    





[35. 85.]


/Users/sean/anaconda/lib/python3.6/site-packages/stingray/utils.py:118: UserWarning: SIMON says: Errorbars on cross spectra are not thoroughly tested. Please report any inconsistencies.
  warnings.warn("SIMON says: {0}".format(message), **kwargs)
150it [00:08, 18.18it/s]
150it [00:07, 20.92it/s]
150it [00:07, 20.16it/s]


[ 85. 160.]


150it [00:09, 16.30it/s]
150it [00:07, 19.96it/s]
150it [00:08, 18.75it/s]


[160. 335.]


150it [00:08, 17.59it/s]
150it [00:07, 20.82it/s]
150it [00:07, 19.47it/s]


[ 335. 1210.]


150it [00:08, 16.97it/s]
150it [00:08, 18.33it/s]
150it [00:07, 19.89it/s]


In [45]:
print(uplim_list)
fig, ax = plt.subplots(figsize=(9,6))
ax.errorbar((keV_list.T[0]+keV_list.T[1])/2, rms_list, xerr = (keV_list.T[0]-keV_list.T[1])/2, yerr = rms_err_list, fmt='none', lw=1.0, uplims=uplim_list)
ax.set_xscale('log')
# plt.yscale('log')
ax.set_ylabel(r'$\mathrm{rms\ (\%)}$')
ax.set_xlabel('Energy (keV)')
ax.set_xticks([3, 4, 5, 6, 7, 8,9, 20, 30, 40, 50], minor=True)
ax.set_xticks([10], minor=False)
ax.set_xticklabels(['3','','','','','','','','','','50'], minor=True)
ax.set_xticklabels(['10'], minor=False)
plt.tight_layout()
plt.savefig(plot_dir + 'rms_E.pdf')
plt.close()

[False, False, False, False]
